<a href="https://colab.research.google.com/github/dsiervo/topnews/blob/all_categories/topNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [3]:
# install ktrain
!pip3 install ktrain

     |████████████████████████████████| 92kB 6.3MB/s 
     |████████████████████████████████| 1.0MB 15.6MB/s 
     |████████████████████████████████| 245kB 34.4MB/s 
     |████████████████████████████████| 1.8MB 49.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.7.3-cp36-none-any.whl size=113951 sha256=789bbd78645c7d476203eefdab66e62ffda71215e5dde62f68d2d3206266c2e3
  Stored in directory: /root/.cache/pip/wheels/86/20/70/000142264c6a768a4e8b67d8ef2ebc32c018684acb2eed7586
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=3f57405c9fa6e4794b5b1fb3d96554dbbe844fb1894a490390a50f015c9355b0
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=f38e2467b45554145d75d095c09764a202500ab7389cd169c6b917d37849c155
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e33286

In [4]:
import ktrain
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
from sklearn.metrics import accuracy_score

using Keras version: 2.2.4-tf


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Data exploration and cleaning

In [6]:
path = '/content/drive/My Drive/News_Category_Dataset_v2.json'

df_ = pd.read_json(path, lines=True)

df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
category             200853 non-null object
headline             200853 non-null object
authors              200853 non-null object
link                 200853 non-null object
short_description    200853 non-null object
date                 200853 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [7]:
df_.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


Se eliminan las columnas que no parecen aportar información valiosa para la clasificación por categoría y se unen en una sola a aquellas que si parecen hacerlo.

In [8]:
#df_['text'] = df_['authors']+' '+df_['headline'] +' '+ df_['short_description']
df_['text'] = df_['headline'] +' '+ df_['short_description']

df = df_.drop(['link', 'date', 'headline', 'short_description', 'authors'], axis=1)

df.head()

,category,text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...


In [9]:
cat_count = df['category'].value_counts()
cat_count

POLITICS          32739
WELLNESS          17827
ENTERTAINMENT     16058
TRAVEL             9887
STYLE & BEAUTY     9649
PARENTING          8677
HEALTHY LIVING     6694
QUEER VOICES       6314
FOOD & DRINK       6226
BUSINESS           5937
COMEDY             5175
SPORTS             4884
BLACK VOICES       4528
HOME & LIVING      4195
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3651
WOMEN              3490
IMPACT             3459
DIVORCE            3426
CRIME              3405
MEDIA              2815
WEIRD NEWS         2670
GREEN              2622
WORLDPOST          2579
RELIGION           2556
STYLE              2254
SCIENCE            2178
WORLD NEWS         2177
TASTE              2096
TECH               2082
MONEY              1707
ARTS               1509
FIFTY              1401
GOOD NEWS          1398
ARTS & CULTURE     1339
ENVIRONMENT        1323
COLLEGE            1144
LATINO VOICES      1129
CULTURE & ARTS     1030
EDUCATION          1004
Name: category, 

In [10]:
#defining data
trace = go.Bar(x = list(cat_count.index),y=df['category'].value_counts())
#defining layout
layout = go.Layout(title='Categories count',xaxis=dict(title='Category'))
#defining figure and plotting
figure = go.Figure(data=[trace],layout=layout)
iplot(figure)

In [11]:
df[df['category'] == 'PARENTING']['text'].iloc[24]

'Finding My Tribe After the Death of My Daughter There is laughter and smiles and happy recollections of our late children, and there are tears and sobs and anguished recollections of our late children.'

In [12]:
df[df['category'] == 'PARENTS']['text'].iloc[24]

'By Letting Go Of Perfection, I Found My Strength As A Mother There is a difference between striving to do our very best for our children, and striving for perfection.'

Uniendo categorías que son prácticamente iguales

In [0]:
culture = ['ARTS & CULTURE', 'ARTS', 'CULTURE & ARTS']
worldpost = ['WORLDPOST', 'THE WORLDPOST']
parents = ['PARENTING', 'PARENTS']

#df['category'][df['category'].isin(culture)] = "CULTURE"
#df['category'][df['category'].isin(worldpost)] = "WORLDPOST"
#df['category'][df['category'].isin(parents)] = "PARENTS"

Se transforman las categorías en números enteros para poder usar librería ktrain


In [0]:
cats = df.category.unique()
def cat_to_num(lab):
  return np.where(cats == lab)[0][0]

In [15]:
df['label'] = df['category'].apply(cat_to_num)
df.head(15)

,category,text,label
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,0
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,1
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...,1
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,1
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,1
5,ENTERTAINMENT,Morgan Freeman 'Devastated' That Sexual Harass...,1
6,ENTERTAINMENT,Donald Trump Is Lovin' New McDonald's Jingle I...,1
7,ENTERTAINMENT,What To Watch On Amazon Prime That’s New This ...,1
8,ENTERTAINMENT,Mike Myers Reveals He'd 'Like To' Do A Fourth ...,1
9,ENTERTAINMENT,What To Watch On Hulu That’s New This Week You...,1


Separamos los datos entre datos de entrenamiento y datos de prueba

In [0]:
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(df['text'], df['label'])


## Feature engineering and downloading pretrained BERT model


In [17]:
(x_train, y_train), (x_test, y_test), preproc = ktrain.text.texts_from_array(
                                                            x_train=X_train.to_list(), y_train=Y_train.to_list(),
                                                            x_test=X_test.to_list(), y_test=Y_test.to_list(),
                                                            class_names=list(cats),
                                                            preprocess_mode='bert',
                                                            maxlen=350, 
                                                            max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


## Model creation and training

In [18]:
tf.device('/device:GPU:0')
model = ktrain.text.text_classifier('bert', (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model,train_data=(x_train, y_train), batch_size=6,
                             workers=-1, use_multiprocessing = True)

Is Multi-Label? False
maxlen is 350
done.


In [19]:
learner.autofit(5e-5, 2)



begin training using triangular learning rate policy with max lr of 5e-05...
Train on 150639 samples
Epoch 1/2
150639/150639 [==============================] - 7579s 50ms/sample - loss: 1.2750 - acc: 0.6406
Epoch 2/2
150639/150639 [==============================] - 7558s 50ms/sample - loss: 0.8621 - acc: 0.7387


In [20]:
learner.validate(val_data=(x_test, y_test), class_names=list(cats))

                precision    recall  f1-score   support

         CRIME       0.71      0.68      0.69       867
 ENTERTAINMENT       0.75      0.82      0.78      3981
    WORLD NEWS       0.50      0.31      0.38       548
        IMPACT       0.52      0.41      0.46       857
      POLITICS       0.80      0.86      0.83      8199
    WEIRD NEWS       0.55      0.47      0.51       685
  BLACK VOICES       0.60      0.54      0.57      1085
         WOMEN       0.52      0.44      0.47       891
        COMEDY       0.65      0.54      0.59      1298
  QUEER VOICES       0.77      0.79      0.78      1551
        SPORTS       0.77      0.82      0.79      1166
      BUSINESS       0.64      0.56      0.60      1481
        TRAVEL       0.83      0.89      0.85      2471
         MEDIA       0.69      0.55      0.61       720
          TECH       0.60      0.59      0.60       508
      RELIGION       0.63      0.66      0.65       622
       SCIENCE       0.67      0.60      0.63  

array([[ 586,   26,    2, ...,    9,    1,    0],
       [   7, 3274,    4, ...,    0,    0,    7],
       [   5,    5,  171, ...,    0,    0,    1],
       ...,
       [   2,    1,    0, ...,  247,    0,    0],
       [   1,    4,    0, ...,    2,  187,    2],
       [   0,   21,    0, ...,    1,    1,  106]])

## Using our trained Bert model

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [22]:
predictor.get_classes()

['CRIME',
 'ENTERTAINMENT',
 'WORLD NEWS',
 'IMPACT',
 'POLITICS',
 'WEIRD NEWS',
 'BLACK VOICES',
 'WOMEN',
 'COMEDY',
 'QUEER VOICES',
 'SPORTS',
 'BUSINESS',
 'TRAVEL',
 'MEDIA',
 'TECH',
 'RELIGION',
 'SCIENCE',
 'LATINO VOICES',
 'EDUCATION',
 'COLLEGE',
 'PARENTS',
 'ARTS & CULTURE',
 'STYLE',
 'GREEN',
 'TASTE',
 'HEALTHY LIVING',
 'THE WORLDPOST',
 'GOOD NEWS',
 'WORLDPOST',
 'FIFTY',
 'ARTS',
 'WELLNESS',
 'PARENTING',
 'HOME & LIVING',
 'STYLE & BEAUTY',
 'DIVORCE',
 'WEDDINGS',
 'FOOD & DRINK',
 'MONEY',
 'ENVIRONMENT',
 'CULTURE & ARTS']

In [23]:
predictor.predict(X_test[0:1])

ValueError: ignored

In [0]:
print(x_test[0])

In [0]:
model_path = '/content/drive/My Drive/NewsBert/'
# let's save the predictor for later use
predictor.save(model_path+'my_predictor')

In [0]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor(model_path+'my_predictor')

In [0]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict(X_test[0:1])

In [0]:
ktrain.text.texts_from_array?